In [1]:
import agents_env
from agents_env.agent_replay_motion import HumanoidReplay
from utils.SimpleConverter import SimpleConverter
from utils.util_data import *
from copy import deepcopy

d:\Python Projects\Thesis\Physis_Based_Branch_Final-Env_14_05_2024\Physics-Based-RL\some_math\transformations.py:32: SyntaxWarning: invalid escape sequence '\*'
  """Homogeneous Transformation Matrices and Quaternions.
d:\Python Projects\Thesis\Physis_Based_Branch_Final-Env_14_05_2024\Physics-Based-RL\some_math\transformations.py:867: SyntaxWarning: invalid escape sequence '\*'
  """Return matrix to transform given vector set into second vector set.


In [2]:

from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import train as ppo
from brax.training.agents.ppo import networks as ppo_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap
import jax.random
from jax import lax

Here I will have my agent

In [3]:
from agents_env.agent_env_template import HumanoidDiff
from some_math.rotation6D import quaternion_to_rotation_6d
from some_math.math_utils_jax import *
from agents_env.losses import *

In [4]:

class HumanoidEnvTrain(HumanoidDiff):
    def __init__(self, reference_trajectory_qpos, 
                 reference_trajectory_qvel, 
                 duration_trajectory,
                 dict_duration, 
                 model_path,
                 kp_gains,
                 kd_gains,
                 reference_x_pos,
                 reference_x_rot,
                 **kwargs):
        super().__init__(reference_trajectory_qpos, 
                         reference_trajectory_qvel, 
                         duration_trajectory, 
                         dict_duration, 
                         model_path, **kwargs)
        self.kp__gains = kp_gains
        self.kd__gains = kd_gains
        #the row lenght is the number of frames of the motion
        #thus it is the lenght, it will be the same with qvel
        self.rollout_lenght = reference_trajectory_qpos.shape[0]
        #I want to save another instance of the model for the refernece 
        self.sys_reference = deepcopy(self.sys)
        self.reference_x_pos = reference_x_pos
        self.reference_x_rot = reference_x_rot

        #for now this will be hardcaode
        self.rot_weight =  0.5
        self.vel_weight =  0.01
        self.ang_weight =  0.01
        self.reward_scaling= 0.02
        #self.err_threshold = 0.4
        self.err_threshold = 1.0
        #for now it will be the same size
        self.cycle_len = reference_trajectory_qpos.shape[0]
  
    
    
    
    #set pd callback
    def set_pd_callback(self,pd_control):
        self.pd_function = pd_control
 
    
    def _demo_replay(self, state,ref_data_pos,current_idx)-> State:
        global_pos_state = state.x.pos
        #jax.debug.print("pos state: {}",global_pos_state)
        #jax.debug.print("pos ref: {}",ref_data_pos)
        error = loss_l2_pos(global_pos_state, ref_data_pos)
        jax.debug.print("error: {}",error)
        
        replay = jp.where(error > self.err_threshold, jp.float32(1), jp.float32(0))
        jax.debug.print("replay: {}",replay)
        #replace the ref_state data
          # Define the true and false branch functions for lax.cond
        def true_fun(_):
            # Update state to reference state and maintain step index
            return self.set_ref_state_pipeline(current_idx)
            #return self.set_ref_step(current_idx,state)
            

        def false_fun(_):
            # Return the original state with updated metrics
            return state
        # Use lax.cond to conditionally switch between states
        new_data = lax.cond(replay == 1, true_fun, false_fun, None)
        
        return new_data,replay
        

    
    def reset(self, rng: jp.ndarray) -> State:
        
        #set this as zero
        reward, done, zero = jp.zeros(3)
        #random state initialization (RSI)
        new_step_idx = jax.random.randint(rng, shape=(), minval=0, maxval=self.rollout_lenght)
        data = self.get_reference_state(new_step_idx)
        metrics = {'step_index': new_step_idx, 'pose_error': zero, 'fall': zero}
        obs = self._get_obs(data, new_step_idx)
        jax.debug.print("obs shape{}", obs.shape)
        
        state = State(data, obs, reward, done, metrics)
        
        #update the replay with 0 index
        state.metrics.update(replay=jp.zeros(1)[0])
        
        return state
    
    
    
    def _get_obs(self, data: mjx.Data, step_idx: jp.ndarray)-> jp.ndarray:
          
        current_step_inx =  jp.asarray(step_idx, dtype=jp.int32)
        #we take out the first index that is the world pos
        current_xpos = data.xpos[1:]
        current_xrot = data.xquat[1:]
        #get rid pf the first index that is the root, we just want
        #pos relative to the root, thus the root will become zero
        relative_pos = (current_xpos - current_xpos[0])[1:].ravel()
        jax.debug.print("relative shape{}", relative_pos.shape)
        
        #this is already in quat form
        current_qpos_root = data.qpos[3:7]
        #qpos of the joins this are scale values, since they are hinge joints
        current_qpos_joints = data.qpos[7:] 
        #now I will convert them into quaterions
        #first the joints that are onedofs, thus axis angle to quaterion
        hinge_quat = self.hinge_to_quat(current_qpos_joints)
        #now get a 13x4 quaterion, were we combine links of 3DOFS
        
        local_quat = self.local_quat(current_qpos_root,current_xrot,hinge_quat,self.one_dofs_joints_idx,self.link_types_array_without_root)
        
        
        #now we convert it to a 6D matrix representation
        local_rot_6D= quaternion_to_rotation_6d(local_quat).ravel()
        
        #remeber for now we have the linear vel of the root
        linear_vel = data.qvel[0:3]
        angular_vel = data.qvel[3:]
        jax.debug.print("linear vel{}", linear_vel.shape)
        jax.debug.print("angular vel{}", angular_vel.shape)
        
        #if needed to calculate the local vel and ang relative to the
        #center of mass
        cvel = Motion(vel=data.cvel[1:, 3:], ang=data.cvel[1:, :3])
        
        
        #get the phi value I will do that later
        phi = ( current_step_inx% self.cycle_len) / self.cycle_len
        phi = jp.asarray(phi)
        #in theory it is mutiable if we do concatenate [] instead of
        #() since, one is a list and the other a tuple
        return jp.concatenate([relative_pos,local_rot_6D,linear_vel,angular_vel,phi[None]])
   
    
    
    #this will grab a reference state from the reference trajectory
    #dont confuse with the state of the main agent
    def set_new_ref_state(self,step_index):
        #grab the current qpos and qvel of the reference
        
        ref_qp = self.reference_trajectory_qpos[step_index]
        ref_qv = self.reference_trajectory_qvel[step_index]
        
        data = mjx.make_data(self.sys_reference)
        data = data.replace(qpos=ref_qp, qvel=ref_qv)
        
        self.new_ref_data = mjx.forward(self.sys_reference, data)
    
    
    def set_ref_step(self,step_index,state):
        ref_qp = self.reference_trajectory_qpos[step_index]
        ref_qv = self.reference_trajectory_qvel[step_index]
        data = state.replace(qpos=ref_qp, qvel=ref_qv)
        return mjx.step(self.sys_reference, data)
        
    
    def set_ref_state_pipeline(self,step_index):
        ref_qp = self.reference_trajectory_qpos[step_index]
        ref_qv = self.reference_trajectory_qvel[step_index]
        #now I will return a state depending on the index and the reference trajectory
        return self._pipeline.init(self.sys_reference, ref_qp, ref_qv, self._debug)
        
    
    
    def get_com_reference(self):
        #here I want to calculate the center of mass
        #we only want the root
        return self.new_ref_data.subtree_com[1]
        
    
    def get_reference_state(self,step_index):
        ref_qp = self.reference_trajectory_qpos[step_index]
        ref_qv = self.reference_trajectory_qvel[step_index]
        #now I will return a state depending on the index and the reference trajectory
        return self.pipeline_init(ref_qp,ref_qv)
        
    #just with a custom target but not selected joints
    def step(self, state: State, action: jp.ndarray,
                                           custom_target,time) -> State:
        
        initial_idx = state.metrics['step_index']
        current_step_inx =  jp.asarray(initial_idx, dtype=jp.int32)
                
        #jax.debug.print("time ref: {}",self.new_ref_data.time)
        current_state_ref = self.set_ref_state_pipeline(current_step_inx)
    
        
        #current qpos and qvel for the torque    
        qpos = state.pipeline_state.q
        qvel = state.pipeline_state.qd
        
        
        #this will be modified by a one without custom target   
        torque = self.pd_function(custom_target,self.sys,state,qpos,qvel,
                                 self.kp__gains,self.kd__gains,time,self.sys.dt) 
        
        data = self.pipeline_step(state.pipeline_state,torque)
        
        #first get the values, first value
        global_pos_state = data.x.pos
        global_rot_state = quaternion_to_rotation_6d(data.x.rot)
        global_vel_state = data.xd.vel
        global_ang_state = data.xd.ang
        #now for the reference trajectory
        global_pos_ref = self.reference_x_pos[current_step_inx]
        global_rot_ref = quaternion_to_rotation_6d(self.reference_x_rot[current_step_inx])
        global_vel_ref = current_state_ref.xd.vel
        global_ang_ref = current_state_ref.xd.ang
        
        reward = -1 * (mse_pos(global_pos_state, global_pos_ref) +
               self.rot_weight * mse_rot(global_rot_state, global_rot_ref) +
               self.vel_weight * mse_vel(global_vel_state, global_vel_ref) +
               self.ang_weight * mse_ang(global_ang_state, global_ang_ref)
               ) * self.reward_scaling
        
        #jax.debug.print("rewards: {}",reward)
        
        #here I will do the fall
        #on the z axis
        fall = jp.where(data.qpos[2] < 0.2, jp.float32(1), jp.float32(0))
        fall = jp.where(data.qpos[2] > 1.7, jp.float32(1), fall)
        
        #jax.debug.print("fall: {}",fall)
        #jax.debug.print("qpos: {}",data.qpos[0:3])
        
        #here the demoreplay
        new_data,replay=self._demo_replay(data,self.reference_x_pos[current_step_inx],current_step_inx)

        data = data.replace(qpos=new_data.qpos, qvel=new_data.qvel, q=new_data.q,qd=new_data.qd,
                            xpos=new_data.xpos, xquat=new_data.xquat,x=new_data.x,xd=new_data.xd)
        #jax.debug.print("data time data: {}",data.time)
        
        
        #get the observations
        obs = self._get_obs(data, current_step_inx)
        
        #increment the step index to know in which episode and wrap
        next_step_index = (current_step_inx + 1) % self.rollout_lenght
        
        state.metrics.update(
            step_index=next_step_index,
            pose_error=loss_l2_relpos(global_pos_state, global_pos_ref),
            fall=fall,
            replay=replay
        )
        
        
        return state.replace(
            pipeline_state= data, obs=obs, reward=reward, done=state.metrics['fall']
        )
        
        
        
    

In [5]:
#since we are going to use custom trajectory, we will set up the initial
#and the end time
t_init = 1
t_end = 3

In [6]:
#this is just dummy data to initialize the agent
trajectory = SimpleConverter('motions/humanoid3d_punch.txt')
trajectory.load_mocap()
model_path = 'models/final_humanoid_no_gravity.xml'

data_mocap_matrix = jp.asarray(trajectory.data)
data_pos_mocap = jp.asarray(trajectory.data_pos)
data_vel_mocap = jp.asarray(trajectory.data_vel)

data_dict_mocap = trajectory.duration_dict

data_xpos_mocap=jp.asarray(trajectory.data_xpos)
data_xrot_mocap=jp.asarray(trajectory.data_xrot)


In [7]:
#get th kp and kd for the agent
kp,kd = generate_kp_kd_gains()
print(kp)
print(kd)

[1000 1000 1000  100  100  100  400  400  400  300  400  400  400  300
  500  500  500  500  400  400  400  500  500  500  500  400  400  400]
[100 100 100  10  10  10  40  40  40  30  40  40  40  30  50  50  50  50
  40  40  40  50  50  50  50  40  40  40]


In [8]:
envs.register_environment('humanoidEnvMimic', HumanoidEnvTrain)
env_name = 'humanoidEnvMimic'
env = envs.get_environment(env_name=env_name,
                           reference_trajectory_qpos=data_pos_mocap,
                           reference_trajectory_qvel = data_vel_mocap,
                            duration_trajectory=trajectory.total_time,
                            dict_duration= data_dict_mocap,
                           model_path=model_path,
                           kp_gains = kp,
                           kd_gains = kd,
                           reference_x_pos=data_xpos_mocap,
                           reference_x_rot=data_xrot_mocap)
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step)


#select the pd_control
from agents_env.pds_controllers_agents import stable_pd_controller_custom_trajectory
env.set_pd_callback(stable_pd_controller_custom_trajectory)

In [9]:
from some_math.math_utils import generate_trajectory,compute_cubic_trajectory,start_trajectories

In [10]:
trajec_dict = dict()


a_jnt_right =get_actuator_indx(env.sys.mj_model,'right_shoulder','Y')
a_jnt_left =get_actuator_indx(env.sys.mj_model,'left_shoulder','X')
a_jnt_left_elbow = get_actuator_indx(env.sys.mj_model,'left_elbow','X')
a_jnt_right_elbow = get_actuator_indx(env.sys.mj_model,'right_elbow','X')

right_knee = get_actuator_indx(env.sys.mj_model,'right_knee','X')
left_knee = get_actuator_indx(env.sys.mj_model,'left_knee','X')

trajec_dict[a_jnt_right] = generate_trajectory(t_init,t_end, 0, -1.5)
#trajectory left
trajec_dict[a_jnt_left] = generate_trajectory(t_init, t_end, 0, 1.5)
# #left elbow   
trajec_dict[a_jnt_left_elbow]= generate_trajectory(t_init, t_end, 0, 1.5)
# #right elbow
trajec_dict[a_jnt_right_elbow] = generate_trajectory(t_init, t_end, 0, 1.5)

trajec_dict[right_knee] = generate_trajectory(t_init, t_end, 0, 0)

trajec_dict[left_knee] = generate_trajectory(t_init, t_end, 0, 0)


start_trajec = start_trajectories(trajec_dict)

In [11]:
print(a_jnt_right)
print(a_jnt_left)
print(a_jnt_left_elbow)
print(a_jnt_right_elbow)
print(right_knee)
print(left_knee)


7
10
13
9
17
24


In [33]:
# initialize the state
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]
# grab a 500 steps
n_steps = 500

relative shape(array(36),)
6D shape(array(78),)
linear vel(array(3),)
angular vel(array(31),)
obs shape(array(149),)


In [13]:
phi = ( 1% env.rollout_lenght) / env.rollout_lenght
phi = jp.asarray(phi)
print(env.rollout_lenght)
print(phi)
print(phi[None].shape)

65
0.015384615
(1,)


In [31]:
print(env.dt)
print(env.sys.dt)

0.016
0.002


In [15]:

for i in range(env.rollout_lenght):
    
    
    ctrl = -0.1 * jp.ones(env.sys.nu)
    #time
    time = state.pipeline_state.time
    
    #print('time: ',state.pipeline_state.time)
    time = jp.clip(time, t_init, t_end)
    
         
    state = jit_step(state, ctrl,start_trajec,time)
    
    rollout.append(state.pipeline_state)

error: 0.0068603502586483955
replay: 0.0
relative shape(array(36),)
6D shape(array(78),)
linear vel(array(3),)
angular vel(array(31),)
error: 0.016810789704322815
replay: 0.0
relative shape(array(36),)
6D shape(array(78),)
linear vel(array(3),)
angular vel(array(31),)
error: 0.03149264305830002
replay: 0.0
relative shape(array(36),)
6D shape(array(78),)
linear vel(array(3),)
angular vel(array(31),)
error: 0.050926461815834045
replay: 0.0
relative shape(array(36),)
6D shape(array(78),)
linear vel(array(3),)
angular vel(array(31),)
error: 0.07869470119476318
replay: 0.0
relative shape(array(36),)
6D shape(array(78),)
linear vel(array(3),)
angular vel(array(31),)
error: 0.1060178205370903
replay: 0.0
relative shape(array(36),)
6D shape(array(78),)
linear vel(array(3),)
angular vel(array(31),)
error: 0.13279953598976135
replay: 0.0
relative shape(array(36),)
6D shape(array(78),)
linear vel(array(3),)
angular vel(array(31),)
error: 0.16033105552196503
replay: 0.0
relative shape(array(36),)


In [16]:
media.show_video(env.render(rollout, camera='back'), fps=1.0/env.dt)

This browser does not support the video tag.


In [17]:
state.pipeline_state.xpos

Array([[0.        , 0.        , 0.        ],
       [1.3056957 , 0.99009013, 0.9648775 ],
       [1.445925  , 0.9916117 , 1.1548793 ],
       [1.5909817 , 0.99682176, 1.3253491 ],
       [1.660793  , 0.8270922 , 1.2968829 ],
       [1.4993786 , 0.7729181 , 1.0812001 ],
       [1.5153122 , 1.1432036 , 1.4110146 ],
       [1.3401104 , 1.1748039 , 1.2016962 ],
       [1.3398627 , 0.9163884 , 0.940251  ],
       [1.2551073 , 0.9586092 , 0.52947736],
       [1.1013926 , 0.9648356 , 0.14957413],
       [1.2715286 , 1.0637919 , 0.989504  ],
       [1.0115696 , 1.0800873 , 0.65805733],
       [0.7266531 , 1.0665992 , 0.36372054]], dtype=float32)

In [18]:
state.pipeline_state.x.rot

Array([[ 0.91781044,  0.07726707,  0.30288747,  0.24477135],
       [ 0.9047812 ,  0.07968156,  0.33609205,  0.24912661],
       [ 0.9317894 ,  0.0572604 ,  0.3192332 ,  0.16303411],
       [ 0.91845965, -0.02868285,  0.32668713,  0.22109914],
       [ 0.9185831 , -0.02859926,  0.32633987,  0.22110997],
       [ 0.92348105,  0.1208991 ,  0.32328132,  0.16749728],
       [ 0.976454  ,  0.1616017 ,  0.06217405,  0.12867332],
       [ 0.9713859 ,  0.07054585,  0.08832066,  0.20888329],
       [ 0.9577433 ,  0.04937205,  0.18471283,  0.21487518],
       [ 0.9573113 ,  0.04863211,  0.19095111,  0.21148945],
       [ 0.9044193 ,  0.11421832,  0.3063038 ,  0.27414984],
       [ 0.8832934 ,  0.09669513,  0.362752  ,  0.28080958],
       [ 0.89037555,  0.08086501,  0.3476158 ,  0.28258774]],      dtype=float32)

Testing how to get the global rotations

In [19]:


# Remove the first character ('f') and convert the rest to a list of integers
link_types_numbers = [int(char) for char in env.sys.link_types[1:]]

# Convert the list of integers to a JAX array
link_types_array_without_root = jp.array(link_types_numbers)


In [20]:
env.sys.jnt_axis

Array([[ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0., -1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0., -1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0., -1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0., -1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]], dtype=float32)

In [21]:
#save the root qpos
current_pos_rot = state.pipeline_state.qpos[0:3]
current_root_rot = state.pipeline_state.qpos[3:7]
current_joints = state.pipeline_state.qpos[7:]

print(state.pipeline_state.qpos)



[ 1.3106205e+00  9.9319774e-01  9.6669102e-01  9.1937596e-01
  7.6894976e-02  3.0486882e-01  2.3641042e-01  1.8550269e-02
  6.6944785e-02  1.2085486e-02  5.4647407e-04 -5.0804105e-02
 -1.7751941e-01 -1.7197482e-01  3.5004403e-02 -1.2389739e-01
  1.1533165e-03  1.1353712e-01 -7.4390419e-02 -1.2196984e-01
  5.3376919e-01 -1.2551783e-01 -4.1884944e-01 -9.0300567e-02
 -1.9520026e-01  1.1529820e-03  1.1743709e-02 -8.2779815e-03
  5.1749263e-02  4.1107847e-03  8.2576156e-02 -1.2193187e-01
 -3.8143620e-02 -2.1993540e-02 -1.0890461e-02]


In [22]:
#now get the indices for the one joint index


joints_one_dofs_indices=jp.array([env.right_elbow_joint,env.left_elbow_joint,env.right_knee_joint,env.left_knee_joint])
print(joints_one_dofs_indices)


[ 9 13 17 24]


In [23]:
def axis_angle_to_quat(axis: jax.Array, angle: jax.Array) -> jax.Array:
  """Provides a quaternion that describes rotating around axis by angle.

  Args:
    axis: (3,) axis (x,y,z)
    angle: () float angle to rotate by

  Returns:
    A quaternion that rotates around axis by angle
  """
  s, c = jp.sin(angle * 0.5), jp.cos(angle * 0.5)
  return jp.insert(axis * s, 0, c)


In [24]:
def quat_mul(u: jax.Array, v: jax.Array) -> jax.Array:
  """Multiplies two quaternions.

  Args:
    u: (4,) quaternion (w,x,y,z)
    v: (4,) quaternion (w,x,y,z)

  Returns:
    A quaternion u * v.
  """
  return jp.array([
      u[0] * v[0] - u[1] * v[1] - u[2] * v[2] - u[3] * v[3],
      u[0] * v[1] + u[1] * v[0] + u[2] * v[3] - u[3] * v[2],
      u[0] * v[2] - u[1] * v[3] + u[2] * v[0] + u[3] * v[1],
      u[0] * v[3] + u[1] * v[2] - u[2] * v[1] + u[3] * v[0],
  ])


In [25]:
#this works properly
right_elbow = current_joints[9]
right_elbow_axis = env.sys.jnt_axis[10]

right_elbow_qloc = axis_angle_to_quat(right_elbow_axis,right_elbow )

quat = quat_mul(state.pipeline_state.x.rot[3], right_elbow_qloc)


quat




Array([ 0.9186479 , -0.02855535,  0.32615742,  0.22111563], dtype=float32)

In [26]:
def compute_quaternion_for_joint(joint_axis,joint_angle):
    
    joint_quat = axis_angle_to_quat(joint_axis,joint_angle)
    return joint_quat

In [27]:

def combine_quaterions_joint_3DOFS(quats):
    # Combine rotations in XYZ order, ensure the multiplication reflects this
    #first element is w the real number
    combined_quat = quat_mul(quat_mul(quats[0], quats[1]),quats[2])
    return combined_quat


In [28]:
#now do the same but with vmap
#jnt_axis without free joint
axis_hinge = env.sys.jnt_axis[1:]

vmap_compute_quaternion = jax.vmap(compute_quaternion_for_joint, in_axes=(0, 0))

hinge_quat = vmap_compute_quaternion(axis_hinge,current_joints) 

print(hinge_quat)
print(hinge_quat.shape)

[[ 9.9995697e-01  9.2750015e-03  0.0000000e+00  0.0000000e+00]
 [ 9.9943984e-01  0.0000000e+00  3.3466142e-02  0.0000000e+00]
 [ 9.9998176e-01  0.0000000e+00  0.0000000e+00  6.0427063e-03]
 [ 9.9999994e-01  2.7323703e-04  0.0000000e+00  0.0000000e+00]
 [ 9.9967736e-01 -0.0000000e+00 -2.5399320e-02 -0.0000000e+00]
 [ 9.9606347e-01 -0.0000000e+00 -0.0000000e+00 -8.8643208e-02]
 [ 9.9630535e-01 -8.5881487e-02 -0.0000000e+00 -0.0000000e+00]
 [ 9.9984682e-01  0.0000000e+00  1.7501308e-02  0.0000000e+00]
 [ 9.9808180e-01 -0.0000000e+00 -0.0000000e+00 -6.1909080e-02]
 [ 9.9999982e-01  0.0000000e+00 -5.7665817e-04  0.0000000e+00]
 [ 9.9838912e-01  5.6738071e-02  0.0000000e+00  0.0000000e+00]
 [ 9.9930835e-01 -0.0000000e+00 -3.7186634e-02 -0.0000000e+00]
 [ 9.9814099e-01 -0.0000000e+00 -0.0000000e+00 -6.0947124e-02]
 [ 9.6459717e-01  0.0000000e+00 -2.6372761e-01  0.0000000e+00]
 [ 9.9803132e-01 -6.2717728e-02 -0.0000000e+00 -0.0000000e+00]
 [ 9.7815067e-01 -0.0000000e+00 -2.0789722e-01 -0.00000

In [29]:
# Mask to filter out the quaternions that are not to be combined in triples
mask = jp.ones(hinge_quat.shape[0], dtype=bool)
mask = mask.at[joints_one_dofs_indices].set(False)



#index 0 since is a tuple and we want the indices that are index 0
combinable_indices = jp.where(mask)[0]
non_combinable_indices = jp.where(~mask)[0]
print(combinable_indices)
no_grouped_quaterions = hinge_quat[non_combinable_indices]


#select and store these indices
# Assuming the remaining quaternions are multiple of three
# Reshape the array to (-1, 3, 4) where 3 is the number of quaternions to be combined and 4 is the quaternion dimension
grouped_quaternions = hinge_quat[combinable_indices].reshape(-1, 3, 4)
#this will be applied on the first axs
vmap_combined_quat = jax.vmap(combine_quaterions_joint_3DOFS)

quat_combined = vmap_combined_quat(grouped_quaternions)

#there are 13 links -12, since we will merge the root at the end the shape 1 is 4 for the quat
quat_loc_all_joints = jp.zeros((state.pipeline_state.x.rot.shape[0]-1,state.pipeline_state.x.rot.shape[1]))

# Create a mask where each position is True if the corresponding link type is 3
link_types_mask = link_types_array_without_root == 3

filter_out_jnt_type_idx = jp.where(link_types_mask)

filter_out_jnt_type_one_dofs = jp.where(~link_types_mask)

quat_loc_all_joints = quat_loc_all_joints.at[filter_out_jnt_type_idx].set(quat_combined)

quat_loc_all_joints = quat_loc_all_joints.at[filter_out_jnt_type_one_dofs].set(no_grouped_quaterions)

quat_loc_all_joints = jp.concatenate([current_root_rot.reshape(1,-1),quat_loc_all_joints],axis=0)


print(quat_loc_all_joints)

# #testing it it works the global chest

# global_quat_chest = quat_mul(state.pipeline_state.x.rot[0],quat_loc_all_joints[1])

# print(global_quat_chest)
#now to 6D rotatio
sixD_matrix = quaternion_to_rotation_6d(quat_loc_all_joints)

# print(sixD_matrix)
# print(sixD_matrix.shape)

[ 0  1  2  3  4  5  6  7  8 10 11 12 14 15 16 18 19 20 21 22 23 25 26 27]
[[ 9.1937596e-01  7.6894976e-02  3.0486882e-01  2.3641042e-01]
 [ 9.9937671e-01  9.4718533e-03  3.3408076e-02  6.3494546e-03]
 [ 9.9574143e-01  2.5235505e-03 -2.5275121e-02 -8.8621520e-02]
 [ 9.9414885e-01 -8.6783104e-02  1.2087169e-02 -6.3171051e-02]
 [ 9.9999982e-01  0.0000000e+00 -5.7665817e-04  0.0000000e+00]
 [ 9.9571532e-01  5.8856193e-02 -3.3602081e-02 -6.2912837e-02]
 [ 9.6459717e-01  0.0000000e+00 -2.6372761e-01  0.0000000e+00]
 [ 9.7581869e-01 -5.1919911e-02 -2.1004537e-01 -3.1036314e-02]
 [ 9.9524087e-01 -0.0000000e+00  9.7445250e-02 -0.0000000e+00]
 [ 9.9997401e-01  5.5217271e-04  5.8741560e-03 -4.1355221e-03]
 [ 9.9880904e-01  2.5934452e-02  9.8506291e-04  4.1315574e-02]
 [ 9.9814212e-01 -0.0000000e+00  6.0928177e-02 -0.0000000e+00]
 [ 9.9974400e-01 -1.9009350e-02 -1.1098223e-02 -5.2341758e-03]]


In [30]:
def update_rotation(local_angle, local_axis):
    # Convert the joint angle and axis to a quaternion
    local_rotation = axis_angle_to_quat(local_axis, local_angle)
    # Apply the local rotation to the parent rotation
    return local_rotation  

# Compute local rotations correctly applying local transformation first
chest_x_qloc = update_rotation(current_joints[0], env.sys.jnt_axis[1])
chest_y_qloc = update_rotation(current_joints[1], env.sys.jnt_axis[2])
chest_z_qloc = update_rotation(current_joints[2], env.sys.jnt_axis[3])

# Combine rotations in XYZ order, ensure the multiplication reflects this
combined_quat = quat_mul(quat_mul(chest_x_qloc, chest_y_qloc), chest_z_qloc)


print(combined_quat)


global_quat_chest = quat_mul(state.pipeline_state.x.rot[0],combined_quat)
print(global_quat_chest)

[0.9993767  0.00947185 0.03340808 0.00634945]
[0.9048334  0.07965811 0.3351888  0.25015882]
